In [1]:
from azureml.core import Workspace, Experiment,ScriptRunConfig

ws = Workspace.get(name="quick-starts-ws-127886")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127886
Azure region: southcentralus
Subscription id: dafd62fe-51d3-4671-af50-6bac2f001b5d
Resource group: aml-quickstarts-127886


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)



Creating new cpu-cluster
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({
        "--max_iter": 100,
        "--C": choice(.5, 1.0)
       
    })

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)   

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory= './',entry_script='train.py',
compute_target = compute_target,
pip_packages=['argparse','pandas','numpy'] )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator = est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=10
                                )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [8]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
joblib.dump(best_run, 'model.joblib')


None


['model.joblib']

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric="normalized_mean_absolute_error",
    training_data=x_train,
    label_column_name="loan",
    n_cross_validations=3)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

automl_run = exp.submit(automl_config, show_output=True)

Running on local machine


In [ ]:
# Retrieve and save your best automl model.
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
### YOUR CODE HERE ###

In [ ]:
best_run, fitted_model = automl_run.get_output()

best_run.register_model(model_path='outputs/udacity_project_automl.pkl', model_name='udacity_project_automl')